# Bayesian DRO

We discuss the DRO models based on misspecified parametric distribution assumptions.

They are often imposed in newsvendor or portfolio loss or regression with gaussian assumptions.


# Frequensit versus Bayesian
## Frequenist

PDRO

## Bayesian

Bayesian-DRO.

We provide additional model type called ``newsvendor`` loss. The default setup is as follows:
$\ell(\theta;(X, Y)) = 3 (Y - \theta)_+ + (\theta - Y)_+$.
In this case, X is the unit vector. That corresponds to exponential gamma conjugate.

That is, when conducting fitting, resampling, we ignore the existence of $X$. 

In [1]:
## Newsvendor Example
import numpy as np
import os
# Set the new path
new_path = "/Users/wty6162/Desktop/DRO_solver/dro"
# Change the current working directory
os.chdir(new_path)
from dro.src.linear_model.chi2_dro import *
from dro.src.linear_model.bayesian_dro import *

from dro.src.data.dataloader_regression import regression_basic

feature_dim = 1
sample_num = 1000
X = [np.ones((sample_num, feature_dim))]
y = [np.random.exponential(2, sample_num)]

reg_model = BayesianDRO(input_dim = feature_dim, model_type = 'newsvendor')
reg_model.update({'posterior_param_num': 1, 'distribution_class': 'Exponential', 'distance_type': 'KL'})
reg_model.fit(X, y, resample = False)

/Users/wty6162/Desktop/DRO_solver/dro/dro/src/linear_model/base.py:43: UserWarning: Unsupported model_type: newsvendor. Default supported types are svm, logistic, ols, lad. Please define your personalized loss.
  warnings.warn(f"Unsupported model_type: {model_type}. Default supported types are svm, logistic, ols, lad. Please define your personalized loss.", UserWarning)


{'theta': [1.3400299889891116], 'b': None}

In [2]:
reg_model.distance_type

'KL'

In [15]:
theta = cp.Variable(1)
b = cp.Variable(1)
eta = cp.Variable()
loss = np.sqrt(1) / np.sqrt(sample_num) * cp.norm(cp.pos(reg_model._cvx_loss(X[0], y[0], theta, b) - eta), 2) + eta
t = cp.Variable()
problem = cp.Problem(cp.Minimize(t), [t >= loss])
problem.solve(solver = 'MOSEK')
print(theta.value)

[1.33596979]


In [2]:
## Regression Example
import numpy as np
import os
# Set the new path
new_path = "/Users/wty6162/Desktop/DRO_solver/dro"
# Change the current working directory
os.chdir(new_path)
from dro.src.linear_model.chi2_dro import *
from dro.src.linear_model.bayesian_dro import *

from dro.src.data.dataloader_regression import regression_basic

feature_dim = 5
X, y = regression_basic(num_samples = 100, d = feature_dim, noise = 1)

reg_model = BayesianDRO(input_dim = feature_dim, model_type = 'ols')

reg_model.update({'posterior_param_num': 5, 'distance_type': 'chi2'})
reg_model.fit(X, y)

sample lool 0
sample lool 1
sample lool 2
sample lool 3
sample lool 4


{'theta': [59.767700832743515,
  97.90700650249167,
  63.999537099064085,
  56.32814220728933,
  35.44050785928378],
 'b': array(1.26951835)}